In [1]:
import pandas as pd
import bamboolib as bam
import csv
import requests
from capstonefunc import api

if __name__ == '__main__':
    crime = api('https://data.boston.gov/api/3/action/datastore_search?resource_id=b973d8cb-eeb2-4e7e-99da-c92938efc9c0&limit=100000')
    print(crime.dtypes)
    crime[['SHOOTING', 'OFFENSE_CODE','MONTH','HOUR','YEAR','INCIDENT_NUMBER']] = crime[['SHOOTING', 'OFFENSE_CODE','MONTH','HOUR','YEAR','INCIDENT_NUMBER']].astype({'SHOOTING' : int, 'OFFENSE_CODE': int,'MONTH' : int,'HOUR': int,'YEAR' : int,'INCIDENT_NUMBER': int})
    crime[['Lat', 'Long','OFFENSE_CODE_GROUP','UCR_PART']] = crime[['Lat', 'Long','OFFENSE_CODE_GROUP','UCR_PART']].astype({'Lat' : float, 'Long' : float,'OFFENSE_CODE_GROUP': float,'UCR_PART': float})
    print(crime.dtypes)


STREET                  object
OFFENSE_DESCRIPTION     object
SHOOTING                 int64
OFFENSE_CODE             int64
DISTRICT                object
REPORTING_AREA          object
OCCURRED_ON_DATE        object
DAY_OF_WEEK             object
MONTH                    int64
HOUR                     int64
Long                   float64
YEAR                     int64
Lat                    float64
INCIDENT_NUMBER          int64
_id                      int64
OFFENSE_CODE_GROUP     float64
UCR_PART               float64
Location                object
dtype: object
STREET                  object
OFFENSE_DESCRIPTION     object
SHOOTING                 int32
OFFENSE_CODE             int32
DISTRICT                object
REPORTING_AREA          object
OCCURRED_ON_DATE        object
DAY_OF_WEEK             object
MONTH                    int32
HOUR                     int32
Long                   float64
YEAR                     int32
Lat                    float64
INCIDENT_NUMBER          

In [ ]:
#data_url = 'https://data.boston.gov/dataset/6220d948-eae2-4e4b-8723-2dc8e67722a3/resource/b973d8cb-eeb2-4e7e-99da-c92938efc9c0/download/tmpomwevvy8.csv'
#crime1 = pd.read_csv(data_url, header=0)
#crime1

In [ ]:
crime

In [ ]:
crime.columns

In [2]:
crime = crime.drop(columns=['OFFENSE_CODE_GROUP','UCR_PART','_id'])

In [62]:
crime

                                                  STREET  \
0                                              CENTRE ST   
1                                          WASHINGTON ST   
2                                             SUDBURY ST   
3                                              GIBSON ST   
4                                              GIBSON ST   
...                                                  ...   
15682                                        BISMARCK ST   
15683                                     NEW SUDBURY ST   
15684  CASTLE CT & E BERKELEY ST\nBOSTON  MA 02118\nU...   
15685  TREMONT ST & OAK ST W\nBOSTON  MA 02116\nUNITE...   
15686                                          SCHOOL ST   

                         OFFENSE_DESCRIPTION  SHOOTING  OFFENSE_CODE DISTRICT  \
0            HARASSMENT/ CRIMINAL HARASSMENT         0          2670       E5   
1            FRAUD - CREDIT CARD / ATM FRAUD         0          1106      E13   
2                         INVESTIGAT

In [3]:
mode = crime['DISTRICT'].mode()[0]
crime['DISTRICT'] = crime['DISTRICT'].fillna(mode)

In [4]:
district_means = crime.groupby('DISTRICT')[['Lat', 'Long']].mean()
crime[['Lat', 'Long']] = crime.groupby('DISTRICT')[['Lat', 'Long']].transform(lambda x: x.fillna(x.mean()))


In [65]:
crime

                                                  STREET  \
0                                              CENTRE ST   
1                                          WASHINGTON ST   
2                                             SUDBURY ST   
3                                              GIBSON ST   
4                                              GIBSON ST   
...                                                  ...   
15682                                        BISMARCK ST   
15683                                     NEW SUDBURY ST   
15684  CASTLE CT & E BERKELEY ST\nBOSTON  MA 02118\nU...   
15685  TREMONT ST & OAK ST W\nBOSTON  MA 02116\nUNITE...   
15686                                          SCHOOL ST   

                         OFFENSE_DESCRIPTION  SHOOTING  OFFENSE_CODE DISTRICT  \
0            HARASSMENT/ CRIMINAL HARASSMENT         0          2670       E5   
1            FRAUD - CREDIT CARD / ATM FRAUD         0          1106      E13   
2                         INVESTIGAT

In [6]:
crime['Location'] = crime.apply(lambda row: f'({row["Lat"]}, {row["Long"]})' if pd.isna(row["Location"]) else row["Location"], axis=1)


In [7]:
crime

                                                  STREET  \
0                                              POPLAR ST   
1                                              CENTRE ST   
2                                          WASHINGTON ST   
3                                             SUDBURY ST   
4                                              GIBSON ST   
...                                                  ...   
15917                                        BISMARCK ST   
15918                                     NEW SUDBURY ST   
15919  CASTLE CT & E BERKELEY ST\nBOSTON  MA 02118\nU...   
15920  TREMONT ST & OAK ST W\nBOSTON  MA 02116\nUNITE...   
15921                                          SCHOOL ST   

                         OFFENSE_DESCRIPTION  SHOOTING  OFFENSE_CODE DISTRICT  \
0                         LARCENY ALL OTHERS         0           619       E5   
1            HARASSMENT/ CRIMINAL HARASSMENT         0          2670       E5   
2            FRAUD - CREDIT CARD / A